# 問題文

## 問題

RNA 文字列に含まれるエキソン（exon）とイントロン（intron）が識別されていれば、  
**イントロン部分を削除し、エキソンを連結**することで、翻訳（タンパク質への変換）に備えた新しい RNA 配列を得ることができます。

---

## 与えられるもの

- DNA 文字列 `s`（長さ最大 1000 塩基対）
- `s` の部分文字列としての複数のイントロン  
- すべての文字列は FASTA 形式で与えられます。

---

## 求めるもの

- `s` からイントロンをすべて削除し、残りの部分（エキソン）を結合
- それを RNA に転写（T → U）
- さらに翻訳して得られる **タンパク質文字列**

※ 与えられるデータに対しては、常に **一意の翻訳結果**が存在します。